#### 使用Scipy求解SVM

In [1]:
from scipy.optimize import minimize
import numpy as np
class SVM:
    def __init__(self,C=0.001):
        self.c=C
        
    def fit(self,X,y):
        y=2*y-1
        m,n=X.shape
        unknow_vector=np.random.randn(m+n+1)
        print(len(unknow_vector))
        self.solver=minimize(type(self).minimize_object,unknow_vector,args=(X,y,self.c),
              constraints=({'type': 'ineq', 'args': (X,y),
                            'fun':lambda unknow_vector,X,y:type(self).constraint1(unknow_vector,X,y)},
                           {'type': 'ineq', 'args': (y,),'fun':lambda unknow_vector,y:type(self).constraint2(unknow_vector,y)}                         
             ))
        self.coef_=self.solver.x[:n]
        self.intercept_=self.solver.x[n]
        self.beta_=self.solver.x[n+1:]
        
        self.support_value=y*(np.dot(X,self.coef_.T)+self.intercept_)+self.beta_
        self.support_=np.where(self.support_value<=1.0001)
        self.support_vectors_=X[self.support_]
           
        return self
    
    def predict(self,X):
        m,n=X.shape
        X_=np.full((m,1),fill_value=1)
        X=np.concatenate([X,X_],axis=1)
        pred=X@self.solver.x[:n+1]
        return (np.sign(pred)+1)/2
        
    
    
    def minimize_object(unknow_vector,X,y,c):
        m,n=X.shape
        w=unknow_vector[:n]  
        b=unknow_vector[n]   
        beta=unknow_vector[n+1:]
        object_value=0.5*np.sum(w**2)+c*np.sum(beta)
        return object_value

    def constraint1(unknow_vector,X,y):
        m,n=X.shape
        w=unknow_vector[:n]
        b=unknow_vector[n]
        beta=unknow_vector[n+1:]
        return y*(X@w.T+b)+beta-1

    def constraint2(unknow_vector,y):
        n=len(y)
        beta=unknow_vector[n+1:]    
        return beta

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X,y=load_breast_cancer().data,load_breast_cancer().target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
my_svm=SVM(C=0.00001).fit(X_train,y_train)
pred=my_svm.predict(X_test)
pred

429


array([0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
       0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
       0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1.,
       0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
       0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0.,
       0.])

In [3]:
accuracy_score(y_test,pred)

0.9298245614035088

In [4]:
my_svm.support_vectors_

array([[1.460e+01, 2.329e+01, 9.397e+01, 6.647e+02, 8.682e-02, 6.636e-02,
        8.390e-02, 5.271e-02, 1.627e-01, 5.416e-02, 4.157e-01, 1.627e+00,
        2.914e+00, 3.301e+01, 8.312e-03, 1.742e-02, 3.389e-02, 1.576e-02,
        1.740e-02, 2.871e-03, 1.579e+01, 3.171e+01, 1.022e+02, 7.582e+02,
        1.312e-01, 1.581e-01, 2.675e-01, 1.359e-01, 2.477e-01, 6.836e-02],
       [1.340e+01, 2.052e+01, 8.864e+01, 5.567e+02, 1.106e-01, 1.469e-01,
        1.445e-01, 8.172e-02, 2.116e-01, 7.325e-02, 3.906e-01, 9.306e-01,
        3.093e+00, 3.367e+01, 5.414e-03, 2.265e-02, 3.452e-02, 1.334e-02,
        1.705e-02, 4.005e-03, 1.641e+01, 2.966e+01, 1.133e+02, 8.444e+02,
        1.574e-01, 3.856e-01, 5.106e-01, 2.051e-01, 3.585e-01, 1.109e-01],
       [1.108e+01, 1.883e+01, 7.330e+01, 3.616e+02, 1.216e-01, 2.154e-01,
        1.689e-01, 6.367e-02, 2.196e-01, 7.950e-02, 2.114e-01, 1.027e+00,
        1.719e+00, 1.399e+01, 7.405e-03, 4.549e-02, 4.588e-02, 1.339e-02,
        1.738e-02, 4.435e-03, 1.324e